In [ ]:
# PANEL FORECASTING PACKAGE
# Files in this single textdoc shown as separate sections. Save each section into its own .py file as named.

# ------------------------- requirements.txt -------------------------
# requirements.txt
# ---------------------------------------------------------------
# pandas>=1.3
# numpy>=1.21
# scikit-learn>=1.0
# matplotlib>=3.4
# seaborn>=0.11
# joblib>=1.0
# xarray>=0.18
# linearmodels>=4.24
# libpysal>=4.6  # optional (pysal spatial tools)

# ------------------------- README.md -------------------------------
# README.md
# ---------------------------------------------------------------
# Panel Forecasting Pipeline
# --------------------------
# This package implements a modular, end-to-end pipeline for panel-data
# forecasting of meteorological variables measured at multiple synoptic stations.
# Files:
#  - data_processing.py
#  - spatial_utils.py
#  - models.py
#  - train_eval.py
#  - utils.py
#  - demo.py
# Usage:
#  1. Save each section of this textdoc to the filename shown in the header.
#  2. install requirements: `pip install -r requirements.txt` (optional packages are fine to skip)
#  3. Run: `python demo.py` to execute a demo on synthetic data.


# ------------------------- utils.py --------------------------------
# utils.py
# ---------------------------------------------------------------
import os
import json
import logging
from pathlib import Path
from typing import Any
import joblib

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("panel_forecast")


def ensure_dir(path: str) -> None:
    Path(path).mkdir(parents=True, exist_ok=True)


def save_pickle(obj: Any, path: str) -> None:
    ensure_dir(os.path.dirname(path) or ".")
    joblib.dump(obj, path)
    logger.info(f"Saved to {path}")


def load_pickle(path: str) -> Any:
    return joblib.load(path)


def save_json(obj: Any, path: str) -> None:
    ensure_dir(os.path.dirname(path) or ".")
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)
    logger.info(f"Saved JSON to {path}")


# --------------------- spatial_utils.py -----------------------------
# spatial_utils.py
# ---------------------------------------------------------------
import numpy as np
import pandas as pd
from typing import Tuple, Optional


def haversine(lon1: float, lat1: float, lon2: float, lat2: float) -> float:
    """Return haversine distance in kilometers."""
    # convert to radians
    lon1, lat1, lon2, lat2 = map(np.radians, (lon1, lat1, lon2, lat2))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0 * c
    return km


def pairwise_distance_matrix(stations: pd.DataFrame) -> pd.DataFrame:
    """Compute pairwise haversine distance matrix from stations dataframe with lon/lat columns."""
    coords = stations[["lon", "lat"]].to_numpy()
    n = coords.shape[0]
    D = np.zeros((n, n))
    for i in range(n):
        lon1, lat1 = coords[i]
        for j in range(n):
            lon2, lat2 = coords[j]
            D[i, j] = haversine(lon1, lat1, lon2, lat2)
    return pd.DataFrame(D, index=stations.index, columns=stations.index)


def inverse_distance_weights(D: pd.DataFrame, max_distance: Optional[float] = None, k_nearest: Optional[int] = None) -> pd.DataFrame:
    """Create inverse-distance row-normalized weight matrix. Optionally threshold by max_distance or keep k nearest neighbors."""
    W = D.copy()
    with np.errstate(divide='ignore'):
        W = 1.0 / W
    np.fill_diagonal(W.values, 0.0)
    if max_distance is not None:
        W = W.where(D <= max_distance, 0.0)
    if k_nearest is not None:
        for i in W.index:
            row = W.loc[i].copy()
            # keep top k
            keep = row.nlargest(k_nearest).index
            mask = ~row.index.isin(keep)
            row[mask] = 0.0
            W.loc[i] = row
    # row normalization
    row_sums = W.sum(axis=1)
    row_sums[row_sums == 0] = 1.0
    W = W.div(row_sums, axis=0)
    return W


def spatial_lag(df_values: pd.DataFrame, W: pd.DataFrame) -> pd.DataFrame:
    """Given df_values indexed by station_id and columns as times (or vice-versa), compute W * values per time column.
    df_values: DataFrame with index=station_id and columns=time or variable.
    Returns DataFrame same shape."""
    # Ensure ordering
    W = W.reindex(index=df_values.index, columns=df_values.index).fillna(0.0)
    return W.values.dot(df_values.values)


# --------------------- data_processing.py ---------------------------
# data_processing.py
# ---------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from typing import Dict, Tuple, List, Optional


def preprocess_base(df: pd.DataFrame, date_col: str = "date", station_col: str = "station_id") -> pd.DataFrame:
    """Parse dates, sort by station and date, and ensure station is category."""
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([station_col, date_col]).reset_index(drop=True)
    df[station_col] = df[station_col].astype("category")
    return df


def impute_stationwise(df: pd.DataFrame, vars_: List[str], station_col: str = "station_id", method: str = "forward_back", max_gap: int = 3, knn_neighbors: int = 5) -> pd.DataFrame:
    """Impute missing values: short gaps per-station forward/backward fill up to max_gap; then KNN across stations for remaining gaps.
    Returns imputed df.
    """
    df = df.copy()
    # forward/backward fill per station
    def fill_short(group):
        for v in vars_:
            # limit the forward/back to max_gap by using group-specific mask
            group[v] = group[v].ffill(limit=max_gap).bfill(limit=max_gap)
        return group

    df = df.groupby(station_col).apply(fill_short).reset_index(drop=True)

    # remaining missing -> per-station median
    for v in vars_:
        med = df.groupby(station_col)[v].transform("median")
        df[v] = df[v].fillna(med)

    # if still missing, use KNN across station/time stacked features
    remaining = df[vars_].isna().any().any()
    if remaining:
        logger.info("Falling back to KNN imputer across all rows")
        imputer = KNNImputer(n_neighbors=knn_neighbors)
        df[vars_] = imputer.fit_transform(df[vars_])
    return df


def remove_impossible_values(df: pd.DataFrame, rules: Dict[str, Tuple[Optional[float], Optional[float]]]) -> pd.DataFrame:
    """Clip or set to NaN values outside allowed ranges. rules: var -> (min, max) where None means no bound."""
    df = df.copy()
    for var, (minv, maxv) in rules.items():
        if var in df.columns:
            if minv is not None:
                df.loc[df[var] < minv, var] = np.nan
            if maxv is not None:
                df.loc[df[var] > maxv, var] = np.nan
    return df


def make_time_features(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    dt = df[date_col]
    df["doy"] = dt.dt.dayofyear
    df["doy_sin"] = np.sin(2 * np.pi * df["doy"] / 365.25)
    df["doy_cos"] = np.cos(2 * np.pi * df["doy"] / 365.25)
    df["month"] = dt.dt.month.astype("category")
    df["weekday"] = dt.dt.weekday.astype("category")
    return df


def make_lag_rolling(df: pd.DataFrame, vars_: List[str], lags: List[int] = None, seasonal_lag: int = 365, windows: List[int] = None, station_col: str = "station_id", date_col: str = "date") -> pd.DataFrame:
    """Create lag and rolling features per station.
    lags: list of lags to create (e.g., [1..7]). windows: list of rolling windows.
    """
    if lags is None:
        lags = list(range(1, 8))
    if windows is None:
        windows = [3, 7, 30]
    df = df.copy()
    df = df.sort_values([station_col, date_col])
    for var in vars_:
        for lag in lags:
            df[f"{var}_lag_{lag}"] = df.groupby(station_col)[var].shift(lag)
        # seasonal lag
        df[f"{var}_lag_seasonal_{seasonal_lag}"] = df.groupby(station_col)[var].shift(seasonal_lag)
        # rolling
        for w in windows:
            df[f"{var}_roll_mean_{w}"] = df.groupby(station_col)[var].transform(lambda x: x.shift(1).rolling(w, min_periods=1).mean())
            df[f"{var}_roll_std_{w}"] = df.groupby(station_col)[var].transform(lambda x: x.shift(1).rolling(w, min_periods=1).std())
    return df


def create_feature_matrix(df: pd.DataFrame, target: str, feature_cols: Optional[List[str]] = None, station_col: str = "station_id", date_col: str = "date") -> Tuple[pd.DataFrame, pd.Series]:
    """Return X, y aligned and drop rows with NaN in target."""
    df = df.copy()
    if feature_cols is None:
        # use all columns except date, station, and target
        exclude = {station_col, date_col, target}
        feature_cols = [c for c in df.columns if c not in exclude]
    X = df[[station_col, date_col] + feature_cols].copy()
    y = df[[station_col, date_col, target]].copy()
    # drop rows where target is NaN
    mask = y[target].notna()
    X = X[mask]
    y = y[mask]
    # set index multiindex for convenience
    X = X.set_index([station_col, date_col])
    y = y.set_index([station_col, date_col])[target]
    return X, y


def fit_scalers(X: pd.DataFrame, scaler: Optional[StandardScaler] = None) -> Tuple[StandardScaler, pd.DataFrame]:
    """Fit a StandardScaler across rows on numeric columns and return scaled DataFrame and scaler."""
    num = X.select_dtypes(include=["number"]).copy()
    scaler = scaler or StandardScaler()
    scaled = scaler.fit_transform(num)
    X_scaled = X.copy()
    X_scaled[num.columns] = scaled
    return scaler, X_scaled


def to_tensor(df: pd.DataFrame, vars_: List[str], station_col: str = "station_id", date_col: str = "date"):
    """Return numpy tensor (n_stations, n_vars, n_times) and time index and station list."""
    df = df.copy()
    stations = df[station_col].cat.categories if pd.api.types.is_categorical_dtype(df[station_col]) else df[station_col].unique()
    times = pd.Index(sorted(df[date_col].unique()))
    n_s = len(stations)
    n_t = len(times)
    n_v = len(vars_)
    tensor = np.full((n_s, n_v, n_t), np.nan, dtype=float)
    s_to_idx = {s: i for i, s in enumerate(stations)}
    t_to_idx = {t: i for i, t in enumerate(times)}
    for _, row in df.iterrows():
        s = row[station_col]
        t = row[date_col]
        if s in s_to_idx and t in t_to_idx:
            i = s_to_idx[s]
            j = t_to_idx[t]
            tensor[i, :, j] = [row.get(v, np.nan) for v in vars_]
    return tensor, list(stations), times


# --------------------- models.py ------------------------------------
# models.py
# ---------------------------------------------------------------
from typing import Any, Dict, List, Tuple
from sklearn.linear_model import ElasticNetCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

try:
    from linearmodels.panel import PanelOLS
    from linearmodels.panel import RandomEffects
    HAVE_LINEARMODELS = True
except Exception:
    HAVE_LINEARMODELS = False


def train_within_station_models(X: pd.DataFrame, y: pd.Series, model_type: str = "rf", per_station: bool = True, random_state: int = 42, **kwargs) -> Dict[str, Any]:
    """Train per-station models. Returns dict station_id -> trained model."""
    models = {}
    stations = X.index.get_level_values(0).unique()
    for s in stations:
        Xs = X.xs(s)
        ys = y.xs(s)
        # drop rows with NaN
        mask = Xs.notna().all(axis=1) & ys.notna()
        Xs = Xs.loc[mask]
        ys = ys.loc[mask]
        if Xs.shape[0] < 10:
            logger.warning(f"Station {s} has too few rows ({Xs.shape[0]}), skipping")
            continue
        if model_type == "enet":
            m = ElasticNetCV(cv=5, random_state=random_state)
        elif model_type == "rf":
            m = RandomForestRegressor(n_estimators=100, random_state=random_state)
        else:
            m = GradientBoostingRegressor(random_state=random_state)
        m.fit(Xs, ys)
        models[str(s)] = m
    return models


def predict_within(models: Dict[str, Any], X: pd.DataFrame) -> pd.Series:
    out = []
    for (s, date), _ in X.iterrows():
        Xrow = X.loc[(s, date)].to_frame().T
        model = models.get(str(s))
        if model is None:
            out.append(np.nan)
        else:
            out.append(float(model.predict(Xrow)[0]))
    return pd.Series(out, index=X.index)


def fit_panel_models(X: pd.DataFrame, y: pd.Series, entity_effects: bool = True, time_effects: bool = False) -> Dict[str, Any]:
    """Fit panel FE and RE models using linearmodels if available. Returns dict of results."""
    res = {}
    if not HAVE_LINEARMODELS:
        logger.warning("linearmodels not installed; falling back to OLS with dummies")
        # fallback: create dummies for station and optionally date
        df = X.reset_index()
        df["_y"] = y.reset_index(drop=True)
        station_dummies = pd.get_dummies(df["station_id"], drop_first=True)
        Xd = pd.concat([df.drop(columns=["station_id", "date"]), station_dummies], axis=1)
        import statsmodels.api as sm
        mdl = sm.OLS(df["_y"], sm.add_constant(Xd)).fit()
        res["ols_with_dummies"] = mdl
        return res

    # prepare panel DataFrame
    df = X.reset_index()
    df["y"] = y.reset_index(drop=True)
    df = df.set_index(["station_id", "date"]).sort_index()
    # drop non-numeric columns
    exog = df.select_dtypes(include=["number"]).drop(columns=["y"], errors='ignore')
    if exog.shape[1] == 0:
        raise ValueError("No numeric regressors available for panel model")
    import statsmodels.api as sm
    exog = sm.add_constant(exog)
    if entity_effects:
        mod = PanelOLS(df["y"], exog, entity_effects=True, time_effects=time_effects, check_rank=False, drop_absorbed=True)
    else:
        mod = RandomEffects(df["y"], exog)
    fit = mod.fit(cov_type='clustered', cluster_entity=True)
    res["fit"] = fit
    return res


def spatial_hybrid_features(df: pd.DataFrame, W: pd.DataFrame, vars_: List[str], station_col: str = "station_id", date_col: str = "date") -> pd.DataFrame:
    """Add spatial lag features W * var_t and W * var_{t-1} into dataframe.
    Assumes df indexed by station,date or columns exist.
    """
    df = df.copy()
    # pivot so rows=station, cols=time
    for var in vars_:
        pv = df.pivot(index=station_col, columns=date_col, values=var)
        lag0 = pd.DataFrame(spatial_lag(pv, W), index=pv.index, columns=pv.columns)
        lag1 = pd.DataFrame(spatial_lag(pv.shift(1, axis=1), W), index=pv.index, columns=pv.columns)
        # melt back
        m0 = lag0.stack().rename(f"{var}_spat_lag0")
        m1 = lag1.stack().rename(f"{var}_spat_lag1")
        df = df.join(m0, on=[station_col, date_col])
        df = df.join(m1, on=[station_col, date_col])
    return df


def stacking_meta_learner(predictions: pd.DataFrame, y: pd.Series, 
                          meta_model=None):
    """Train a meta-learner on base model predictions."""
    if meta_model is None:
        from sklearn.linear_model import RidgeCV
        meta_model = RidgeCV(alphas=np.logspace(-3, 3, 7))
    
    # Handle NaN predictions (e.g., if some base models missing output)
    X = predictions.fillna(0)   # simple choice, or use predictions.mean()
    mask = y.notna() & X.notna().all(axis=1)
    
    meta_model.fit(X.loc[mask], y.loc[mask])
    return meta_model


# --------------------- train_eval.py -------------------------------
# train_eval.py
# ---------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit


def rolling_origin_splits(df: pd.DataFrame, n_splits: int = 3, initial_window: int = 365, step: int = 90, date_col: str = "date", station_col: str = "station_id"):
    """Yield train/val index masks for time-aware CV. This simple implementation uses global time index."""
    times = pd.Index(sorted(df[date_col].unique()))
    starts = [times[0] + pd.Timedelta(days=int(step * i)) for i in range(n_splits)]
    for i in range(n_splits):
        start = starts[i]
        train_end = start + pd.Timedelta(days=initial_window)
        val_end = train_end + pd.Timedelta(days=step)
        train_mask = df[date_col] <= train_end
        val_mask = (df[date_col] > train_end) & (df[date_col] <= val_end)
        yield train_mask, val_mask


def evaluate_predictions(y_true: pd.Series, y_pred: pd.Series) -> Dict[str, float]:
    mask = y_true.notna() & y_pred.notna()
    if mask.sum() == 0:
        return {"rmse": np.nan, "mae": np.nan, "r2": np.nan}
    y_true = y_true.loc[mask]
    y_pred = y_pred.loc[mask]
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)),
    }


def plot_obs_vs_pred(y_true: pd.Series, y_pred: pd.Series, station: str, out_path: str = "plots"):
    ensure_dir(out_path)
    df = pd.concat([y_true.rename('obs'), y_pred.rename('pred')], axis=1)
    df = df.sort_index(level=1)  # sort by date
    plt.figure(figsize=(12, 4))
    plt.plot(df.index.get_level_values(1), df['obs'], label='obs')
    plt.plot(df.index.get_level_values(1), df['pred'], label='pred')
    plt.title(f"Observed vs Predicted - {station}")
    plt.legend()
    p = f"{out_path}/obs_pred_{station}.png"
    plt.savefig(p, bbox_inches='tight')
    plt.close()
    logger.info(f"Saved plot {p}")


def plot_residuals_acf(resid: pd.Series, station: str, lags: int = 30, out_path: str = "plots"):
    ensure_dir(out_path)
    from statsmodels.graphics.tsaplots import plot_acf
    plt.figure()
    plot_acf(resid.dropna(), lags=lags)
    p = f"{out_path}/resid_acf_{station}.png"
    plt.savefig(p, bbox_inches='tight')
    plt.close()
    logger.info(f"Saved plot {p}")


def residuals_spatial_heatmap(residuals: pd.Series, stations_meta: pd.DataFrame, out_path: str = "plots"):
    """Create heatmap of pairwise correlations of residuals across stations."""
    ensure_dir(out_path)
    # residuals index: station,date
    pivot = residuals.reset_index().pivot(index='date', columns='station_id', values=0)
    corr = pivot.corr()
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=False)
    p = f"{out_path}/resid_spatial_corr.png"
    plt.title("Residual correlation across stations")
    plt.savefig(p, bbox_inches='tight')
    plt.close()
    logger.info(f"Saved plot {p}")


import matplotlib.pyplot as plt
import os

def plot_station_predictions(y_true: pd.Series, y_pred: pd.Series, station_ids: pd.Series,
                             outdir="plots/stations", max_stations=5):
    os.makedirs(outdir, exist_ok=True)
    df = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "station": station_ids})
    
    for i, (st, grp) in enumerate(df.groupby("station")):
        if i >= max_stations:  # فقط n تا ایستگاه اول برای جلوگیری از شلوغی
            break
        plt.figure(figsize=(10, 5))
        plt.plot(grp.index, grp["y_true"], label="Observed", lw=2)
        plt.plot(grp.index, grp["y_pred"], label="Predicted", lw=2)
        plt.title(f"Station {st} - Observed vs Predicted")
        plt.xlabel("Date")
        plt.ylabel("Target")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"{st}_timeseries.png"))
        plt.close()


def plot_station_scatter(y_true: pd.Series, y_pred: pd.Series, station_ids: pd.Series,
                         outdir="plots/scatter", max_stations=5):
    os.makedirs(outdir, exist_ok=True)
    df = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "station": station_ids})
    
    for i, (st, grp) in enumerate(df.groupby("station")):
        if i >= max_stations:
            break
        plt.figure(figsize=(6, 6))
        plt.scatter(grp["y_true"], grp["y_pred"], alpha=0.6)
        lims = [min(grp["y_true"].min(), grp["y_pred"].min()),
                max(grp["y_true"].max(), grp["y_pred"].max())]
        plt.plot(lims, lims, 'r--')  # خط 1:1
        plt.title(f"Station {st} - Pred vs Obs")
        plt.xlabel("Observed")
        plt.ylabel("Predicted")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"{st}_scatter.png"))
        plt.close()


from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import pandas as pd

def evaluate_by_station(y_true: pd.Series, y_pred: pd.Series, station_ids: pd.Series):
    results = []
    df = pd.DataFrame({"y_true": y_true, "y_pred": y_pred, "station": station_ids})
    
    for st, grp in df.groupby("station"):
        r2 = r2_score(grp["y_true"], grp["y_pred"])
        rmse = np.sqrt(mean_squared_error(grp["y_true"], grp["y_pred"]))
        mae = mean_absolute_error(grp["y_true"], grp["y_pred"])
        results.append({"station": st, "R2": r2, "RMSE": rmse, "MAE": mae})
    
    return pd.DataFrame(results)

# --------------------- demo.py -------------------------------------
# demo.py
# ---------------------------------------------------------------
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# We'll import functions from above as if they were real modules
# In practice, save each chunk to separate files and use `from data_processing import ...`


def synthetic_data_generator(n_stations: int = 10, years: int = 3, seed: int = 42, missing_frac: float = 0.02) -> pd.DataFrame:
    """Generate synthetic daily meteorological panel data with seasonal cycles and spatially correlated noise."""
    rng = np.random.RandomState(seed)
    start = datetime(2018, 1, 1)
    end = start + timedelta(days=365 * years)
    dates = pd.date_range(start, end, freq='D')
    station_ids = [f"S{i:02d}" for i in range(n_stations)]
    # random station metadata
    lats = rng.uniform(-60, 60, size=n_stations)
    lons = rng.uniform(-180, 180, size=n_stations)
    elev = rng.uniform(0, 3000, size=n_stations)
    rows = []
    for si, s in enumerate(station_ids):
        lat = lats[si]
        lon = lons[si]
        amp = 10 + (abs(lat) / 90) * 10  # lat-dependent amplitude
        phase = rng.uniform(0, 2 * np.pi)
        for d in dates:
            doy = d.timetuple().tm_yday
            seasonal = amp * np.sin(2 * np.pi * doy / 365.25 + phase)
            base_temp = 15 + seasonal - elev[si] * 0.0065
            t_mean = base_temp + rng.normal(scale=3.0)
            t_max = t_mean + abs(rng.normal(scale=2.0))
            t_min = t_mean - abs(rng.normal(scale=2.0))
            precip = np.abs(rng.exponential(scale=1.0) * (0.3 + 0.7 * rng.binomial(1, 0.1)))
            wind = np.abs(rng.normal(3, 1.5))
            rh = np.clip(50 + 20 * np.sin(2 * np.pi * doy / 365.25 + phase / 2) + rng.normal(0, 10), 0, 100)
            rows.append({
                'date': d,
                'station_id': s,
                'lat': lat,
                'lon': lon,
                'elevation': elev[si],
                't_mean': t_mean,
                't_max': t_max,
                't_min': t_min,
                'precip': precip,
                'wind': wind,
                'rh': rh
            })
    df = pd.DataFrame(rows)
    # introduce missingness
    mask = rng.rand(len(df)) < missing_frac
    df.loc[mask, 't_max'] = np.nan
    df['station_id'] = df['station_id'].astype('category')
    return df


# config
cfg = {
    'target': 't_max',
    'vars': ['t_mean', 't_min', 'precip', 'wind', 'rh'],
    'lags': list(range(1, 8)),
    'windows': [3, 7, 30],
    'seasonal_lag': 365,
    'horizon': 1,
    'out_dir': 'demo_output'
}

df = synthetic_data_generator(n_stations=6, years=2, missing_frac=0.02)
# from data_processing import preprocess_base, remove_impossible_values, make_time_features, make_lag_rolling, impute_stationwise, create_feature_matrix, fit_scalers, to_tensor
# from spatial_utils import pairwise_distance_matrix, inverse_distance_weights
# from models import train_within_station_models, predict_within, fit_panel_models, spatial_hybrid_features, stacking_meta_learner
# from train_eval import evaluate_predictions, plot_obs_vs_pred
# preprocess
df = preprocess_base(df)
df = remove_impossible_values(df, rules={'precip': (0.0, None)})
df = impute_stationwise(df, vars_=['t_mean', 't_max', 't_min', 'precip', 'wind', 'rh'])
df = make_time_features(df)
df = make_lag_rolling(df, vars_=cfg['vars'] + [cfg['target']], lags=cfg['lags'], seasonal_lag=cfg['seasonal_lag'], windows=cfg['windows'])
# feature matrix
X, y = create_feature_matrix(df, target=cfg['target'])
scaler, Xs = fit_scalers(X)
# spatial
stations_meta = df.drop_duplicates('station_id').set_index('station_id')[[ 'lon', 'lat', 'elevation' ]]
D = pairwise_distance_matrix(stations_meta.reset_index().rename(columns={'index':'station_id'}).set_index('station_id'))
W = inverse_distance_weights(D, k_nearest=3)

# add spatial hybrid features
df_spat = spatial_hybrid_features(df, W, vars_=['t_mean', 't_min', 'precip'])
Xs_spat, y_spat = create_feature_matrix(df_spat, target=cfg['target'])

# train within-station models
within_models = train_within_station_models(Xs, y, model_type='rf')
y_pred_within = predict_within(within_models, Xs)
metrics_within = evaluate_predictions(y, y_pred_within)
logger.info(f"Within-station metrics: {metrics_within}")

# panel model
panel_res = fit_panel_models(Xs, y, entity_effects=True)

if 'fit' in panel_res:
    panel_pred = panel_res['fit'].predict()
    if isinstance(panel_pred, pd.DataFrame) and panel_pred.shape[1] == 1:
        panel_pred = panel_pred.iloc[:, 0]  # flatten to Series
else:
    panel_pred = pd.Series(np.nan, index=y.index)

# stacking
preds_df = pd.DataFrame({'within': y_pred_within, 'panel': panel_pred})
meta = stacking_meta_learner(preds_df, y)
y_meta = pd.Series(meta.predict(preds_df.fillna(0)), index=preds_df.index)
metrics_meta = evaluate_predictions(y, y_meta)
logger.info(f"Stacked metrics: {metrics_meta}")



# plots for a sample station
sample_station = Xs.index.get_level_values(0).unique()[0]
sample_idx = Xs.index[Xs.index.get_level_values(0) == sample_station]
plot_obs_vs_pred(y.loc[sample_idx], y_meta.loc[sample_idx], station=str(sample_station))
# save
ensure_dir(cfg['out_dir'])
save_pickle(within_models, f"{cfg['out_dir']}/within_models.joblib")
save_pickle(scaler, f"{cfg['out_dir']}/scaler.joblib")
logger.info("Demo complete")



C:\Users\psh2610\AppData\Local\Temp\ipykernel_15880\1703540536.py:168: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(station_col).apply(fill_short).reset_index(drop=True)
C:\Users\psh2610\AppData\Local\Temp\ipykernel_15880\1703540536.py:168: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(station_col).apply(fill_short).reset_index(drop=True)
C:\Users\psh2610\AppData\Local\Temp\ipykernel_15880\1703540536.py:172: FutureWarning: The default of observed=False is deprecated and will be ch

CategoricalIndex(['S00', 'S00', 'S00', 'S00', 'S00', 'S00', 'S00', 'S00',
                  'S00', 'S00',
                  ...
                  'S05', 'S05', 'S05', 'S05', 'S05', 'S05', 'S05', 'S05',
                  'S05', 'S05'],
                 categories=['S00', 'S01', 'S02', 'S03', 'S04', 'S05'], ordered=False, dtype='category', name='station_id', length=4386)


In [30]:
# Per-station metrics and plots
stations = Xs.index.get_level_values(0).unique()
all_metrics = {}
for s in stations:
    mask = Xs.index.get_level_values(0) == s
    metrics_within = evaluate_predictions(y.loc[mask], y_pred_within.loc[mask])
    # metrics_panel = evaluate_predictions(y.loc[mask], panel_pred.loc[mask])
    metrics_stacked = evaluate_predictions(y.loc[mask], y_meta.loc[mask])
    all_metrics[s] = {
        'within': metrics_within,
        # 'panel': metrics_panel,
        'stacked': metrics_stacked
    }
    logger.info(f"Station {s} metrics: {all_metrics[s]}")
    # Plot stacked predictions
    plot_obs_vs_pred(y.loc[mask], y_meta.loc[mask], station=str(s), out_path=cfg['out_dir'])

logger.info("Demo complete with per-station metrics and plots.")


INFO:panel_forecast:Station S00 metrics: {'within': {'rmse': 1.0801473622815871, 'mae': 0.7403780259153616, 'r2': 0.9845056868008393}, 'stacked': {'rmse': 1.0824208774646247, 'mae': 0.7416228717047224, 'r2': 0.9844403926931815}}
INFO:panel_forecast:Saved plot demo_output/obs_pred_S00.png
INFO:panel_forecast:Station S01 metrics: {'within': {'rmse': 1.1792221090842063, 'mae': 0.8105770426985969, 'r2': 0.9898354770492443}, 'stacked': {'rmse': 1.1790499303377147, 'mae': 0.8070071695391753, 'r2': 0.9898384450856123}}
INFO:panel_forecast:Saved plot demo_output/obs_pred_S01.png
INFO:panel_forecast:Station S02 metrics: {'within': {'rmse': 1.1593711404119988, 'mae': 0.8063171241051879, 'r2': 0.9865079035906327}, 'stacked': {'rmse': 1.1555445021477029, 'mae': 0.8026663857827719, 'r2': 0.9865968210498269}}
INFO:panel_forecast:Saved plot demo_output/obs_pred_S02.png
INFO:panel_forecast:Station S03 metrics: {'within': {'rmse': 1.1585547724443055, 'mae': 0.8400301205475635, 'r2': 0.9821492734175205}

In [20]:
s = 'S00'
idx = station_ids == s
idx

array([ True,  True,  True, ..., False, False, False], shape=(4386,))

In [26]:
panel_pred

station_id  date      
S00         2019-01-01    6.293710
            2019-01-02    4.436503
            2019-01-03    3.693666
            2019-01-04   -0.539088
            2019-01-05   -1.145989
                            ...   
S05         2019-12-28   -6.016113
            2019-12-29   -6.501794
            2019-12-30   -4.303673
            2019-12-31   -4.262841
            2020-01-01   -6.537705
Name: fitted_values, Length: 2196, dtype: float64

In [27]:
# station ids
station_ids = Xs.index.get_level_values(0)

# حلقه برای هر ایستگاه
for s in station_ids.unique():
    idx = station_ids == s
    # Within-station
    metrics_within_s = evaluate_predictions(y[idx], y_pred_within[idx])
    # Panel
    metrics_panel_s = evaluate_predictions(y[idx], panel_pred[idx])
    # Stacked
    metrics_meta_s = evaluate_predictions(y[idx], y_meta[idx])
    print(f"Station {s}:")
    print(f"  Within: {metrics_within_s}")
    print(f"  Panel:  {metrics_panel_s}")
    print(f"  Stacked: {metrics_meta_s}")

    # رسم نمودارها
    plot_obs_vs_pred(y[idx], y_pred_within[idx], station=str(s), out_path="plots/within")
    plot_obs_vs_pred(y[idx], panel_pred[idx], station=str(s), out_path="plots/panel")
    plot_obs_vs_pred(y[idx], y_meta[idx], station=str(s), out_path="plots/stacked")

IndexError: Boolean index has wrong length: 4386 instead of 2196